# Máquina de soporte vectorial para clasificación binaria

<img style="float: right;" src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b5/Svm_separating_hyperplanes_%28SVG%29.svg/512px-Svm_separating_hyperplanes_%28SVG%29.svg.png" width="300" height="300" >

## Introducción
La máquina de soporte vectorial (**SVM** por sus siglas en inglés Support Vector Machine) es un algoritmo de aprendizaje supervisado el cuál es ampliamente usado para problemas de clasificación en *Machine Learning*.

La SVM fue introducida por Vladimir Vapnik y Alexey Chervonenkis en 1963 como un clasificador lineal para datos linealmente separables. En 1995 se formuló la extensión de éste clasificador al caso no-separable y tener tolerancia en la clasificación de datos. 

Este mismo año se desarolló la técnica para el caso no lineal. Para lograr esto, los datos de entrada son representados en una dimensión mayor donde pueden ser separados linealmente por un hiperplano.

## Formulación del problema
### Producto punto como regla de decisión

El producto punto de dos vectores tiene algunas propiedades geométricas interesantes. Considere dos vectores $x, \, y \in \mathbb{R}^n$, dónde $x_i$ es la $i$-ésima coordenada de $x$; el producto punto es definido por:

$$\langle x,y \rangle = x_1y_1 + \ldots + x_ny_n$$

Ésta fórmula, aunque simple, genera una característica muy interesante, la cuál es importante en *machine learning*, y es una regla de decisión muy conveniente.

En particular, suponga que estamos en el plano Euclidiano, y se tiene una línea $L$ que pasa a través del origen, con un vector unitario $w$ perpendicular a L. Vector **normal** a la línea $L$.

<img style="float: center;" src="https://jeremykun.files.wordpress.com/2017/05/decision-rule-1.png" width="300" height="300" >

Si se toma cualquier vector $x$, entonces el producto punto $\langle x, w \rangle$ es positivo si $x$ está del mismo lado que $w$ respecto a $L$, y negativo en caso contrario. El producto punto es cero, sí y sólo sí $x$ está exactamente en la línea $L$ (paralela), incluso cuando $x$ es el vector cero.

<img style="float: center;" src="https://jeremykun.files.wordpress.com/2017/05/decision-rule-2.png" width="500" height="800" >

El principal hecho para que esta propiedad funcione, es que el producto punto nos da un indicio de *cómo* un vector es proyectado en otro. Cuando se proyecta un vector $x$ en otro vector $w$, ésta proyección es la componente de $x$ que apunta en la misma dirección de $w$.

<img style="float: center;" src="https://jeremykun.files.wordpress.com/2017/05/decision-rule-3-e1495339586386.png" width="300" height="500" >



<http://j2kun.github.io/decision-rule/index.html>